# Tools - Ferramentas

Ferramentas são interfaces que um agente, cadeia ou LLM podem usar para interagir com o mundo. Elas combinam algumas coisas:
- O nome da ferramenta
- Uma descrição do que é a ferramenta
- Esquema JSON dos inputs (entradas) da ferramenta
- A função a ser chamada
- Se o resultado de uma ferramenta deve ser retornado diretamente para o usuário

É útil ter todas essas informações porque essas informações podem ser usadas para construir sistemas que tomam ações! O nome, a descrição e o esquema JSON podem ser usados para solicitar ao LLM que ele saiba como especificar qual ação tomar, e então a função a ser chamada é equivalente a tomar essa ação.

Quanto mais simples for a entrada de uma ferramenta, mais fácil será para um LLM conseguir usá-la. Importante ressaltar que o nome, a descrição e o esquema JSON (se utilizados) são todos utilizados no prompt. Portanto, é muito importante que eles sejam claros e descrevam exatamente como a ferramenta deve ser usada. Pode ser necessário alterar o nome padrão, a descrição ou o esquema JSON se o LLM não estiver entendendo como usar a ferramenta.

## Criando tools com o decorador @tools

In [1]:
from  langchain.agents import tool

@tool
def retorna_temperatura_atual(local: str):
    """Faz uma busca online de temperatura de uma localidade"""
    return '25°C'

retorna_temperatura_atual

StructuredTool(name='retorna_temperatura_atual', description='Faz uma busca online de temperatura de uma localidade', args_schema=<class 'langchain_core.utils.pydantic.retorna_temperatura_atual'>, func=<function retorna_temperatura_atual at 0x0000028BC1559940>)

In [2]:
retorna_temperatura_atual.name

'retorna_temperatura_atual'

In [3]:
retorna_temperatura_atual.description

'Faz uma busca online de temperatura de uma localidade'

In [4]:
retorna_temperatura_atual.args

{'local': {'title': 'Local', 'type': 'string'}}

### Descrevendo os argumentos

In [7]:
from pydantic import BaseModel, Field
from langchain.agents import tool

class RetornaTempArgs(BaseModel):
    local: str = Field(description='Local a ser buscado', examples=['São Paulo', 'Porto Alegre', 'Florianópolis'])

@tool(args_schema=RetornaTempArgs)

def retorna_temperatura_atual(local: str):
    """Faz uma busca online de temperatura de um local"""
    return '25°C'

retorna_temperatura_atual

StructuredTool(name='retorna_temperatura_atual', description='Faz uma busca online de temperatura de um local', args_schema=<class '__main__.RetornaTempArgs'>, func=<function retorna_temperatura_atual at 0x0000028BD3CE20C0>)

In [8]:
retorna_temperatura_atual.args

{'local': {'description': 'Local a ser buscado',
  'examples': ['São Paulo', 'Porto Alegre', 'Florianópolis'],
  'title': 'Local',
  'type': 'string'}}

### Chamando a tool

In [9]:
retorna_temperatura_atual.invoke({'local': 'Porto Alegre'})

'25°C'

## Criando tool com StructuredTool

Outra forma de criar uma tool sem o decorador é utilizando a metaclasse StructuredTool do LangChain. As funcionalidades são bem similares, então você pode utilizar um ou outro dependendo da sua preferência.

In [10]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

class RetornaTempArgs(BaseModel):
    local: str = Field(description='Local a ser buscado', examples=['São Paulo', 'Porto Alegre', 'Florianópolis'])

def retorna_temperatura_atual(local: str):
    return '25°C'

tool_temp = StructuredTool.from_function(
    func=retorna_temperatura_atual,
    name='ToolTemperatura',
    args_schema=RetornaTempArgs,
    description="Faz uma busca online de temperatura de um local",
    return_direct=True
)

tool_temp

StructuredTool(name='ToolTemperatura', description='Faz uma busca online de temperatura de um local', args_schema=<class '__main__.RetornaTempArgs'>, return_direct=True, func=<function retorna_temperatura_atual at 0x0000028BD3D78540>)

In [11]:
tool_temp.name

'ToolTemperatura'

In [13]:
tool_temp.args

{'local': {'description': 'Local a ser buscado',
  'examples': ['São Paulo', 'Porto Alegre', 'Florianópolis'],
  'title': 'Local',
  'type': 'string'}}

In [14]:
tool_temp.description

'Faz uma busca online de temperatura de um local'

In [15]:
tool_temp.invoke({'local': 'Porto Alegre'})

'25°C'